In [1]:
import os
import glob
import html
import io
import re
import time
from pypdf import PdfReader, PdfWriter
from azure.core.credentials import AzureKeyCredential
from azure.storage.blob import BlobServiceClient
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import *
from azure.search.documents import SearchClient
from azure.ai.formrecognizer import DocumentAnalysisClient

from dotenv import load_dotenv
load_dotenv()

connect_str = os.getenv('AZURE_STORAGE_CONNECTION_STRING')

In [6]:
def blob_name_from_file_page(filename, page = 0):
        if os.path.splitext(filename)[1].lower() == ".pdf":
            return os.path.splitext(os.path.basename(filename))[0] + f"-{page}" + ".pdf"
        else:
            return os.path.basename(filename)

def upload_blobs(filename,container,connect_str,verbose:str='True'):
    # connect_str = os.getenv('AZURE_STORAGE_CONNECTION_STRING')
    blob_service =  BlobServiceClient.from_connection_string(connect_str)
    blob_container = blob_service.get_container_client(container)
    if not blob_container.exists():
        blob_container.create_container()

    # if file is PDF split into pages and upload each page as a separate blob
    if os.path.splitext(filename)[1].lower() == ".pdf":
        reader = PdfReader(filename)
        pages = reader.pages
        for i in range(len(pages)):
            blob_name = blob_name_from_file_page(filename, i)
            if verbose: print(f"\tUploading blob for page {i} -> {blob_name}")
            f = io.BytesIO()
            writer = PdfWriter()
            writer.add_page(pages[i])
            writer.write(f)
            f.seek(0)
            blob_container.upload_blob(blob_name, f, overwrite=True)
    else:
        blob_name = blob_name_from_file_page(filename)
        with open(filename,"rb") as data:
            blob_container.upload_blob(blob_name, data, overwrite=True)

In [2]:
def blob_name_from_file_page(filename, page = 0):
        if os.path.splitext(filename)[1].lower() == ".pdf":
            return os.path.splitext(os.path.basename(filename))[0] + f"-{page}" + ".pdf"
        else:
            return os.path.basename(filename)
        
def upload_blobs(filename, container, connect_str, verbose: str = 'True'):
    # connect_str = os.getenv('AZURE_STORAGE_CONNECTION_STRING')
    blob_service = BlobServiceClient.from_connection_string(connect_str)
    blob_container = blob_service.get_container_client(container)
    if not blob_container.exists():
        blob_container.create_container()

    # Find all PDF files matching the filename pattern
    pdf_files = glob.glob(filename)
    for pdf_file in pdf_files:
        reader = PdfReader(pdf_file)
        pages = reader.pages
        for i in range(len(pages)):
            blob_name = blob_name_from_file_page(pdf_file, i)
            if verbose: print(f"\tUploading blob for page {i} of file {pdf_file} -> {blob_name}")
            f = io.BytesIO()
            writer = PdfWriter()
            writer.add_page(pages[i])
            writer.write(f)
            f.seek(0)
            blob_container.upload_blob(blob_name, f, overwrite=True)


In [3]:

upload_blobs('../data/*.pdf','docsksw',connect_str)

	Uploading blob for page 0 of file ../data\캐롯 해외여행보험 약관.pdf -> 캐롯 해외여행보험 약관-0.pdf
	Uploading blob for page 1 of file ../data\캐롯 해외여행보험 약관.pdf -> 캐롯 해외여행보험 약관-1.pdf
	Uploading blob for page 2 of file ../data\캐롯 해외여행보험 약관.pdf -> 캐롯 해외여행보험 약관-2.pdf
	Uploading blob for page 3 of file ../data\캐롯 해외여행보험 약관.pdf -> 캐롯 해외여행보험 약관-3.pdf
	Uploading blob for page 4 of file ../data\캐롯 해외여행보험 약관.pdf -> 캐롯 해외여행보험 약관-4.pdf
	Uploading blob for page 5 of file ../data\캐롯 해외여행보험 약관.pdf -> 캐롯 해외여행보험 약관-5.pdf
	Uploading blob for page 6 of file ../data\캐롯 해외여행보험 약관.pdf -> 캐롯 해외여행보험 약관-6.pdf
	Uploading blob for page 7 of file ../data\캐롯 해외여행보험 약관.pdf -> 캐롯 해외여행보험 약관-7.pdf


In [ ]:
# if args.removeall:
#     remove_blobs(None)
#     remove_from_index(None)
# else:
#     if not args.remove:
#         create_search_index()
    
#     print(f"Processing files...")
#     for filename in glob.glob(args.files):
#         if args.verbose: print(f"Processing '{filename}'")
#         if args.remove:
#             remove_blobs(filename)
#             remove_from_index(filename)
#         elif args.removeall:
#             remove_blobs(None)
#             remove_from_index(None)
#         else:
#             if not args.skipblobs:
#                 upload_blobs(filename)
#             page_map = get_document_text(filename)
#             sections = create_sections(os.path.basename(filename), page_map)
#             index_sections(os.path.basename(filename), sections)